In [ ]:
import pandas as pd
import random
import copy
import matplotlib.pyplot as plt

In [ ]:
df_alimentos = pd.read_csv("alimentos.csv")
df_alimentos.fillna(0, inplace=True)

In [ ]:
METAS_NUTRICIONAIS = {
    "Energia..kcal.": 2000,
    "Proteína..g.": 75,
    "Carboidrato..g.": 275,
    "Lipídeos..g.": 70,
}

TAMANHO_POPULACAO = 30
NUM_GERACOES = 100
NUM_ELITISTAS = 2
TAXA_MUTACAO = 0.1
TAXA_ELITISMO = 0.1
TAMANHO_TORNEIO = 3

SEED_RANDOM = 42
random.seed(SEED_RANDOM)

In [ ]:
def gerar_cardapio_aleatorio(df):
    alimentos = df["Descrição dos alimentos"].dropna().unique().tolist()

    cardapio = {
        "cafe": random.sample(alimentos, k=random.randint(2, 3)),
        "almoco": random.sample(alimentos, k=random.randint(2, 3)),
        "jantar": random.sample(alimentos, k=random.randint(2, 3)),
    }

    return cardapio

populacao_inicial = [gerar_cardapio_aleatorio(df_alimentos) for _ in range(TAMANHO_POPULACAO)]

In [ ]:
def calcular_nutrientes_cardapio(cardapio, df):
    total = {
        "Energia..kcal.": 0,
        "Proteína..g.": 0,
        "Carboidrato..g.": 0,
        "Lipídeos..g.": 0,
    }

    for refeicao, alimentos in cardapio.items():
        for alimento in alimentos:
            dados = df[df["Descrição dos alimentos"] == alimento]
            if not dados.empty:
                for chave in total.keys():
                    valor = dados[chave].values[0]
                    if pd.notna(valor):
                        total[chave] += valor

    return total

In [ ]:
def calcular_fitness(nutrientes, metas):
    fitness = 0

    for chave in metas.keys():
        valor_real = nutrientes.get(chave, 0)
        valor_meta = metas[chave]
        desvio = abs(valor_real - valor_meta)

        fitness -= desvio

    return fitness

In [ ]:
def selecao_por_torneio(populacao, fitnesses, k=3):
    selecionados = []

    for _ in range(len(populacao)):
        participantes = random.sample(list(zip(populacao, fitnesses)), k)
        vencedor = max(participantes, key=lambda x: x[1])
        selecionados.append(vencedor[0])

    return selecionados

In [ ]:
def crossover(card1, card2):
    filho = {}

    for refeicao in ["cafe", "almoco", "jantar"]:
        lista_escolhida = random.choice([card1[refeicao], card2[refeicao]])
        filho[refeicao] = lista_escolhida.copy()

    return filho

In [ ]:
def mutacao(cardapio, df, taxa_mutacao=TAXA_MUTACAO):
    novo_cardapio = copy.deepcopy(cardapio)

    for refeicao in novo_cardapio:
        if random.random() < taxa_mutacao:
            alimentos_possiveis = (
                df["Descrição dos alimentos"].dropna().unique().tolist()
            )
            if novo_cardapio[refeicao]:
                idx = random.randint(0, len(novo_cardapio[refeicao]) - 1)
                novo_cardapio[refeicao][idx] = random.choice(alimentos_possiveis)

    return novo_cardapio

In [ ]:
def algoritmo_genetico(
    populacao_inicial=populacao_inicial,
    df=df_alimentos,
    metas=METAS_NUTRICIONAIS,
    tamanho_populacao=TAMANHO_POPULACAO,
    num_geracoes=NUM_GERACOES
):
    populacao = copy.deepcopy(populacao_inicial)
    historico_fitness = []

    for g in range(num_geracoes):
        fitnesses = [
            calcular_fitness(calcular_nutrientes_cardapio(c, df), metas)
            for c in populacao
        ]

        # Guardar o melhor fitness da geração
        historico_fitness.append(max(fitnesses))

        nova_populacao = []
        selecionados = selecao_por_torneio(populacao, fitnesses)

        for i in range(0, tamanho_populacao, 2):
            pai1 = selecionados[i]
            pai2 = selecionados[i + 1 if i + 1 < tamanho_populacao else 0]
            filho1 = crossover(pai1, pai2)
            filho2 = crossover(pai2, pai1)
            nova_populacao.extend([mutacao(filho1, df), mutacao(filho2, df)])

        populacao = nova_populacao[:tamanho_populacao]

    fitnesses = [
        calcular_fitness(calcular_nutrientes_cardapio(c, df), metas) for c in populacao
    ]
    melhor_idx = fitnesses.index(max(fitnesses))

    return populacao[melhor_idx], fitnesses[melhor_idx], historico_fitness

In [ ]:
melhor_cardapio, melhor_fitness, historico_fitness_base = algoritmo_genetico()

print("\nMelhor cardápio:")
for r, itens in melhor_cardapio.items():
    print(f"{r}: {'; '.join(itens)}")

nutrientes = calcular_nutrientes_cardapio(melhor_cardapio, df_alimentos)
print("\nNutrientes:")
for n, v in nutrientes.items():
    print(f"{n}: {v:.2f}")

print(f"\nFitness final: {melhor_fitness:.2f}")

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(range(1, NUM_GERACOES + 1), historico_fitness_base, color="red", linestyle="-", linewidth=2, label="Sem Elitismo")
plt.xlabel("Geração")
plt.ylabel("Melhor Fitness")
plt.title("Evolução do Melhor Fitness por Geração")
plt.grid(True, alpha=0.3)
plt.show()

In [ ]:
def algoritmo_genetico_com_elitismo(
    populacao_inicial=populacao_inicial,
    df=df_alimentos,
    metas=METAS_NUTRICIONAIS,
    tamanho_populacao=TAMANHO_POPULACAO,
    num_geracoes=NUM_GERACOES,
):
    populacao = copy.deepcopy(populacao_inicial)
    historico_fitness = []

    for g in range(num_geracoes):
        fitnesses = [
            calcular_fitness(calcular_nutrientes_cardapio(c, df), metas)
            for c in populacao
        ]

        # Guardar o melhor fitness da geração
        historico_fitness.append(max(fitnesses))

        # Elitismo: encontrar o melhor indivíduo da geração atual
        melhor_idx = fitnesses.index(max(fitnesses))
        elite = copy.deepcopy(populacao[melhor_idx])  # Cópia profunda do elite

        nova_populacao = []
        selecionados = selecao_por_torneio(populacao, fitnesses)

        # Gerar N-1 novos indivíduos (deixando espaço para o elite)
        for i in range(0, tamanho_populacao - 1, 2):
            pai1 = selecionados[i]
            pai2 = selecionados[i + 1 if i + 1 < tamanho_populacao else 0]
            filho1 = crossover(pai1, pai2)
            filho2 = crossover(pai2, pai1)
            nova_populacao.extend([mutacao(filho1, df), mutacao(filho2, df)])

        # Se N for par, nova_populacao terá N-1 ou N-2 indivíduos, então ajustamos:
        nova_populacao = nova_populacao[: tamanho_populacao - 1]

        # Adiciona o elite à nova população
        nova_populacao.append(elite)

        populacao = nova_populacao[:tamanho_populacao]

    fitnesses = [
        calcular_fitness(calcular_nutrientes_cardapio(c, df), metas) for c in populacao
    ]
    melhor_idx = fitnesses.index(max(fitnesses))

    return populacao[melhor_idx], fitnesses[melhor_idx], historico_fitness

In [ ]:
melhor_cardapio, melhor_fitness, historico_fitness_elitismo = algoritmo_genetico_com_elitismo()

print("\nMelhor cardápio:")
for r, itens in melhor_cardapio.items():
    print(f"{r}: {'; '.join(itens)}")

nutrientes = calcular_nutrientes_cardapio(melhor_cardapio, df_alimentos)
print("\nNutrientes:")
for n, v in nutrientes.items():
    print(f"{n}: {v:.2f}")

print(f"\nFitness final: {melhor_fitness:.2f}")

In [ ]:
plt.figure(figsize=(10, 6))

plt.plot(range(1, NUM_GERACOES + 1), historico_fitness_base, color="red", linestyle="-", linewidth=2, label="Sem Elitismo")
plt.plot(range(1, NUM_GERACOES + 1), historico_fitness_elitismo, color="blue", linestyle="-", linewidth=2, label="Com Elitismo")
plt.legend()
plt.xlabel("Geração")
plt.ylabel("Melhor Fitness")
plt.title("Evolução do Melhor Fitness por Geração")
plt.grid(True, alpha=0.3)
plt.show()